## Measuring the latent representations: activations of the message functions

In [45]:
import sys
import os

parent_dir = os.path.abspath(os.path.join(os.getcwd(), '..'))
sys.path.append(parent_dir)

from model import *
from sklearn.model_selection import train_test_split
import pandas as pd
import numpy as np
import torch
from torch.utils.data import DataLoader, TensorDataset

In [46]:
#load the data
data = load_data('../simulations/datasets/r1_n=3_dim=2_nt=1000_dt=0.005')
X, y = data

#make train, test, val sets
X_train, X_test, y_train, y_test = train_test_split(X, y, shuffle=False)
X_test, X_val, y_test, y_val = train_test_split(X_test, y_test, shuffle=False)

In [47]:
cutoff = 10000
val_cutoff = 2500

#train the model on a small amount of data
model = train((X_train[:cutoff], y_train[:cutoff]), (X_val[:cutoff], y_val[:cutoff]), 25)

Epoch: 1/25: 100%|██████████| 313/313 [00:02<00:00, 124.26it/s]


training loss: 0.9895, val loss: 1.6207


Epoch: 2/25: 100%|██████████| 313/313 [00:01<00:00, 204.11it/s]


training loss: 0.7332, val loss: 1.4217


Epoch: 3/25: 100%|██████████| 313/313 [00:01<00:00, 176.78it/s]


...

training loss: 0.2012, val loss: 0.6841


Epoch: 24/25: 100%|██████████| 313/313 [00:02<00:00, 106.10it/s]


training loss: 0.1931, val loss: 0.5807


Epoch: 25/25: 100%|██████████| 313/313 [00:03<00:00, 78.34it/s]


training loss: 0.2214, val loss: 0.5969


In [60]:
TensorDataset(X_test)

In [92]:
dataloader = DataLoader(TensorDataset(X_test[:1]), batch_size=1, shuffle=False)
edge_index = get_edge_index(X_test.shape[1])
#print(edge_index)
tmp = 0
for (nodes,) in dataloader:
    s1 = nodes[:, edge_index[0]]
    #print(s1)
    s2 = nodes[:, edge_index[1]]
    edge_features = torch.cat((s1, s2), dim=-1)
    messages = model.edge_model(edge_features)
    print(messages.shape)
    batch_messages = torch.cat((s1, s2, messages), dim=-1)
    print(batch_messages.shape)
    

torch.Size([1, 6, 100])
torch.Size([1, 6, 112])


In [ ]:
def get_message_features(model, input_data, batch_size=32):
    """
    Make pd.Dataframe of messages and particle information
    
    Args:
        model (NBodyGNN): Trained model
        input_data (torch.Tensor): Input data with shape [no_timesteps, no_nodes, node_features]
        batch_size (int): Size of batches to process
        
    Returns:
        pd.DataFrame: DataFrame containing node features and message information
    """

    model.eval()  
    edge_index = get_edge_index(input_data.shape[1])
    
    # Create dataloader
    dataset = TensorDataset(input_data)
    dataloader = DataLoader(dataset, batch_size=batch_size, shuffle=False)
    
    all_message_info = []
    
    with torch.no_grad():
        for (nodes,) in dataloader:
            x_i = nodes[:, edge_index[0]]  # Source nodes
            x_j = nodes[:, edge_index[1]]  # Target nodes
            
            x = torch.cat((x_i, x_j), dim=-1)
            messages = model.edge_model(x) #put thru edge model
            #messages shape is [batch_size, num_edges, 100]
            
            #combine node features with messages along final axis
            message_info = torch.cat((x_i, x_j, messages), dim=-1)
            
            #reshape batch_messages to be 2D for pandas: [batch_size * num_edges, features]
            #batch info and edge info doesnt matter. only related feature info and latent messages
            message_info = message_info.reshape(-1, message_info.shape[-1])
            all_message_info.append(message_info)
    
    # Combine all batches
    message_features = torch.cat(all_message_info, dim=0)
    
    # Convert to numpy for DataFrame creation
    message_features = message_features.numpy()
    
    # Create column names
    node_info = ['x', 'y', 'delta_x', 'delta_y', 'q', 'm'] #for 2d
    source_cols = [f'{f}1' for f in node_info]
    target_cols = [f'{f}2' for f in node_info]
    message_cols = [f'e{i}' for i in range(messages.shape[-1])]
    columns = source_cols + target_cols + message_cols
    
    # Create DataFrame
    message_info = pd.DataFrame(message_features, columns=columns)
    
    # distances between particles
    message_info['x_dist'] = message_info.x1 - message_info.x2
    message_info['y_dist'] = message_info.y1 - message_info.y2
    message_info['r'] = np.sqrt(message_info.x_dist**2 + message_info.y_dist**2)
    
    # Calculate relative velocities
    message_info['dvx'] = message_info.delta_x1 - message_info.delta_x2
    message_info['dvy'] = message_info.delta_y1 - message_info.delta_y2
    message_info['v_rel'] = np.sqrt(message_info.dvx**2 + message_info.dvy**2)
    
    return message_info

In [103]:
message_features = get_message_features(model, X_test[:500])
message_features

,x1,y1,delta_x1,delta_y1,q1,m1,x2,y2,delta_x2,delta_y2,...,e96,e97,e98,e99,x_dist,y_dist,r,dvx,dvy,v_rel
0,1.108470,-3.027177,-0.107625,-0.693829,-0.871233,1.801894,1.019855,-3.755330,0.474216,-1.642119,...,-0.046022,0.096742,-0.184472,0.265260,0.088615,0.728153,0.733526,-0.581841,0.948290,1.112561
1,1.108470,-3.027177,-0.107625,-0.693829,-0.871233,1.801894,2.941403,-0.552212,2.070446,-1.332116,...,-0.019179,-0.046165,0.066972,0.071643,-1.832933,-2.474966,3.079789,-2.178071,0.638288,2.269670
2,1.019855,-3.755330,0.474216,-1.642119,-1.288934,1.142944,1.108470,-3.027177,-0.107625,-0.693829,...,-0.232747,-0.458506,0.470316,0.406838,-0.088615,-0.728153,0.733526,0.581841,-0.948290,1.112561
3,1.019855,-3.755330,0.474216,-1.642119,-1.288934,1.142944,2.941403,-0.552212,2.070446,-1.332116,...,-0.015118,-0.045306,0.060357,0.068809,-1.921548,-3.203119,3.735280,-1.596229,-0.310002,1.626053
4,2.941403,-0.552212,2.070446,-1.332116,0.595053,0.623231,1.108470,-3.027177,-0.107625,-0.693829,...,0.093666,0.022980,-0.046237,0.199105,1.832933,2.474966,3.079789,2.178071,-0.638288,2.269670
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2995,1.490004,0.604459,0.959115,1.252741,0.495583,1.296262,0.835605,0.783813,-0.987951,-0.009257,...,0.285503,-0.021025,0.002730,0.270177,0.654399,-0.179354,0.678532,1.947065,1.261998,2.320281
2996,-1.636888,1.912824,-0.466934,0.984240,1.361227,0.673946,1.490004,0.604459,0.959115,1.252741,...,-0.096031,-0.058165,0.050241,0.090458,-3.126891,1.308365,3.389582,-1.426049,-0.268502,1.451106
2997,-1.636888,1.912824,-0.466934,0.984240,1.361227,0.673946,0.835605,0.783813,-0.987951,-0.009257,...,-0.114815,-0.054776,0.035625,0.088639,-2.472492,1.129011,2.718066,0.521016,0.993497,1.121826
2998,0.835605,0.783813,-0.987951,-0.009257,-1.039976,1.101690,1.490004,0.604459,0.959115,1.252741,...,-0.493235,-0.202894,0.138584,0.229762,-0.654399,0.179354,0.678532,-1.947065,-1.261998,2.320281


In [100]:
message_columns = [col for col in message_features.columns if col.startswith('e')]
message_columns

['e0',
 'e1',
 'e2',
 'e3',
 'e4',
 'e5',
 'e6',
 'e7',
 'e8',
 'e9',
 'e10',
 'e11',
 'e12',
 'e13',
 'e14',
 'e15',
 'e16',
 'e17',
 'e18',
 'e19',
 'e20',
 'e21',
 'e22',
 'e23',
 'e24',
 'e25',
 'e26',
 'e27',
 'e28',
 'e29',
 'e30',
 'e31',
 'e32',
 'e33',
 'e34',
 'e35',
 'e36',
 'e37',
 'e38',
 'e39',
 'e40',
 'e41',
 'e42',
 'e43',
 'e44',
 'e45',
 'e46',
 'e47',
 'e48',
 'e49',
 'e50',
 'e51',
 'e52',
 'e53',
 'e54',
 'e55',
 'e56',
 'e57',
 'e58',
 'e59',
 'e60',
 'e61',
 'e62',
 'e63',
 'e64',
 'e65',
 'e66',
 'e67',
 'e68',
 'e69',
 'e70',
 'e71',
 'e72',
 'e73',
 'e74',
 'e75',
 'e76',
 'e77',
 'e78',
 'e79',
 'e80',
 'e81',
 'e82',
 'e83',
 'e84',
 'e85',
 'e86',
 'e87',
 'e88',
 'e89',
 'e90',
 'e91',
 'e92',
 'e93',
 'e94',
 'e95',
 'e96',
 'e97',
 'e98',
 'e99']